In [1]:
import glob
import pandas as pd
from pathlib import Path
from xml.dom.minidom import Document
import cv2

In [2]:
dict = {'0': "0",  # 字典对类型进行转换
        '1': "1",
        '2': "2",
        '3': "3",
        '4': "4",
       '5': '5',
       '6': "6",
       '7': "7",
       '8': "8",}

In [3]:
txtPath = "../watoo20k/watoo_anomaly_hard20k_labelme/check_txt/txt/"
picPath = "../watoo20k/watoo_anomaly_hard20k_labelme/check_txt/check_img_txt/"
xmlPath = "../watoo20k/watoo_anomaly_hard20k_labelme/check_txt/gen_xml/"

In [ ]:
makexml("../watoo20k/watoo_anomaly_hard20k_labelme/check_txt/txt/",
        "../watoo20k/watoo_anomaly_hard20k_labelme/check_txt/gen_xml/",
        "../watoo20k/watoo_anomaly_hard20k_labelme/check_txt/check_img_txt/")

In [4]:
def makexml(txtPath, xmlPath, picPath):  # 读取txt路径，xml保存路径，数据集图片所在路径
#     files = os.listdir(txtPath)
    pathes = glob.glob(txtPath+'*.txt')
    files = [Path(x).name for x in pathes]
    for i, name in tqdm(enumerate(files)):
        xmlBuilder = Document()
        annotation = xmlBuilder.createElement("annotation")  # 创建annotation标签
        xmlBuilder.appendChild(annotation)
        txtFile = open(txtPath+name)
        txtList = txtFile.readlines()
        img = cv2.imread(picPath+name[0:-4]+".jpg")
        Pheight, Pwidth, Pdepth=img.shape
        flag = 0
        for i in txtList:
            flag += 1
            oneline = i.strip().split(" ")
            folder = xmlBuilder.createElement("folder")  # folder标签
            folderContent = xmlBuilder.createTextNode("VOC2007")
            folder.appendChild(folderContent)
            annotation.appendChild(folder)
            if flag == 1:
                filename = xmlBuilder.createElement("filename")  # filename标签
                filenameContent = xmlBuilder.createTextNode(name[0:-4]+".jpg")
                filename.appendChild(filenameContent)
                annotation.appendChild(filename)

                size = xmlBuilder.createElement("size")  # size标签
                width = xmlBuilder.createElement("width")  # size子标签width
                widthContent = xmlBuilder.createTextNode(str(Pwidth))
                width.appendChild(widthContent)
                size.appendChild(width)
                height = xmlBuilder.createElement("height")  # size子标签height
                heightContent = xmlBuilder.createTextNode(str(Pheight))
                height.appendChild(heightContent)
                size.appendChild(height)
                depth = xmlBuilder.createElement("depth")  # size子标签depth
                depthContent = xmlBuilder.createTextNode(str(Pdepth))
                depth.appendChild(depthContent)
                size.appendChild(depth)
                annotation.appendChild(size)

            object = xmlBuilder.createElement("object")
            picname = xmlBuilder.createElement("name")
            nameContent = xmlBuilder.createTextNode(dict[oneline[0]])
            picname.appendChild(nameContent)
            object.appendChild(picname)
            pose = xmlBuilder.createElement("pose")
            poseContent = xmlBuilder.createTextNode("Unspecified")
            pose.appendChild(poseContent)
            object.appendChild(pose)
            truncated = xmlBuilder.createElement("truncated")
            truncatedContent = xmlBuilder.createTextNode("0")
            truncated.appendChild(truncatedContent)
            object.appendChild(truncated)
            difficult = xmlBuilder.createElement("difficult")
            difficultContent = xmlBuilder.createTextNode("0")
            difficult.appendChild(difficultContent)
            object.appendChild(difficult)
            bndbox = xmlBuilder.createElement("bndbox")
            xmin = xmlBuilder.createElement("xmin")
            mathData=int(((float(oneline[1]))*Pwidth+1)-(float(oneline[3]))*0.5*Pwidth)
            xminContent = xmlBuilder.createTextNode(str(mathData))
            xmin.appendChild(xminContent)
            bndbox.appendChild(xmin)
            ymin = xmlBuilder.createElement("ymin")
            mathData = int(((float(oneline[2]))*Pheight+1)-(float(oneline[4]))*0.5*Pheight)
            yminContent = xmlBuilder.createTextNode(str(mathData))
            ymin.appendChild(yminContent)
            bndbox.appendChild(ymin)
            xmax = xmlBuilder.createElement("xmax")
            mathData = int(((float(oneline[1]))*Pwidth+1)+(float(oneline[3]))*0.5*Pwidth)
            xmaxContent = xmlBuilder.createTextNode(str(mathData))
            xmax.appendChild(xmaxContent)
            bndbox.appendChild(xmax)
            ymax = xmlBuilder.createElement("ymax")
            mathData = int(((float(oneline[2]))*Pheight+1)+(float(oneline[4]))*0.5*Pheight)
            ymaxContent = xmlBuilder.createTextNode(str(mathData))
            ymax.appendChild(ymaxContent)
            bndbox.appendChild(ymax)
            object.appendChild(bndbox)

            annotation.appendChild(object)

        f = open(xmlPath+name[0:-4]+".xml", 'w')
        xmlBuilder.writexml(f, indent='\t', newl='\n', addindent='\t', encoding='utf-8')
        f.close()